In [20]:
#numpy, pandas, streamlitのパッケージインストール
import numpy as np
import pandas as pd
!pip install streamlit

!pip install gensim
from google.colab import drive
import re
import csv
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [21]:
#テキスト入力
!pip install streamlit
!pip install jedi

import streamlit as st
import time
from PIL import Image

app3.pyへ

#そのまま

形態素解析

In [13]:
!apt install aptitude
!aptitude install mecab libmecab-dev mecab-ipadic-utf8 git make curl xz-utils file -y
!pip install mecab-python3==0.7

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
aptitude is already the newest version (0.8.13-3ubuntu1).
0 upgraded, 0 newly installed, 0 to remove and 24 not upgraded.
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the requested version (2.7.0-20070801+main-3)
git is already installed at the requested version (1:2.34.1-1ubuntu1.10)
make is already installed at the requested version (4.3-4.1build1)
curl is already installed at the requested version (7.81.0-1ubuntu1.15)
xz-utils is already installed at the requested version (5.2.5-2ubuntu1)
file is already installed at the requested version (1:5.41-3ubuntu0.1)
mecab is already installed at the requested version (0.996-14build9)
libmecab-dev is already installed at the requested version (0.996-14build9)
mecab-ipadic-utf8 is already installed at the re

In [22]:
!pip install janome

上の f = f.readlines()[2:]でー2して文章が空になってた可能性有。

app3.pyへ

形態素解析

x_Chart N
C
ベ
ン
ダ

学習用データ準備

モデル作成

model保存
#モデル置き換え

#streamlitに印字されている？

文書比較

app3.pyへ

TypeError: can only join an iterable

#app3.pyへ

#関数化

TypeError: unsupported operand type(s) for *: 'float' and 'NoneType'

chart

ローカルダウンロード

#この中に!pip install streamlitがなくともipynbに記載があればOK

In [23]:
%%writefile app4.py
# %% [markdown]
# # スケジュール算出プログラム

# %% [markdown]
# ## 1.ライブラリのインポート
# pandasは表の処理

# %%
import random
import datetime
import numpy as np
import pandas as pd
from array import array
import locale                               # ローカライズ用パッケージ

# %% [markdown]
# ## 2.時間のローカライズ

# %%
locale.setlocale(locale.LC_TIME, "ja_JP")   # 日本にローカライズ（これがないと、英語の曜日が出えてくる）

# %%
def read_excels():
    #excel_path = "f_iden_3.xlsm" #xlsx→xlsm
    excel_path = "'/content/drive/MyDrive/data/f_iden_3.xlsm"
    off_day_df  = pd.read_excel(excel_path, sheet_name="yukyu")                         # 休みの希望表
    skill_df    = pd.read_excel(excel_path, sheet_name="skill", header=[0, 1])          # スキル表
    due_date_df = pd.read_excel(excel_path, sheet_name="calendar")                      # 納期表
    start_day:pd.Series = pd.read_excel(excel_path, sheet_name="start")

    # 空データ(NaN)の置換(0)
    off_day_df.fillna(0, inplace=True)
    # skill_df.fillna(0, inplace=True)
    due_date_df.fillna(0, inplace=True)
    # データの置換
    off_day_df.replace("◎", 2, inplace=True)                                            # 希望休（◎）を 2 に置換
    due_date_df.replace("☆", 3, inplace=True)                                          # 納期（☆）を 3 に置換
    # 列ごとのデータ型(dtype)を、intに変更しておく
    off_day_df[off_day_df.columns[2:]] = off_day_df[off_day_df.columns[2:]].astype("int64")
    due_date_df[due_date_df.columns[5:]] = due_date_df[due_date_df.columns[5:]].astype("int64")   #スキルグループ→工程による列追加

    # 品番の抽出
    product_num = due_date_df.loc[:, "品番"]

    # 土日の削除
    # データの中から土日の日付を取り出す
    off_days = [date for date in due_date_df.columns[5:] if date.strftime("%a") in "土日"]  #スキルグループ→工程による列追加

    # 土日の日付を削除する
    off_day_df.drop(columns=off_days, inplace=True)
    due_date_df.drop(columns=off_days, inplace=True)

    return off_day_df, skill_df, due_date_df, product_num , start_day

# %%
off_day_df, skill_df, due_date_df, product_num , start_day = read_excels()

# %%


# %%


# %%


# %% [markdown]
# ## 3.第1世代関数

# %%
def first_gen(off_day_df: pd.DataFrame, skill_df: pd.DataFrame, due_date_df: pd.DataFrame, product_num: pd.Series):

    # 調整後の納期表（納期表のコピー）　→　以降、『調整納期表』　と呼ぶ
    adjusted_due_date_df = due_date_df.copy()
    # 作業者の割り振り　と　作業者の希望休を調整納期表にコピー
    worker_list = []
    # indexごとにrowを取り出して、作業者をランダムに割り振る
    for index, row in adjusted_due_date_df.iterrows():
#作業者を選ぶ →ajauted_date_dfのスキルグループ番号と同じskill_dfのスキルグループ番号のリストを選択、worker選択。()はマルチヘッダー。
        #workers: pd.DataFrame = skill_df.loc[:, (["工程"], [row["工程"]])]　#そのまま
        workers: pd.DataFrame = skill_df.loc[:, (["スキルグループ"], [row["スキルグループ"]])] #スキルグループ→工程関係ない
        worker = workers.dropna().sample().values[0, 0]
#練習
        #print('選ばれたスキルグループworkers',workers)
        #print('スキルグループの任意のworker',worker)
        worker_list.append(worker)
        worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]), off_day_df.columns[2:]]                   # 作業者の希望休を抽出
        #worker_off_days: pd.DataFrame = off_day_df.loc[off_day_df["設備"].isin([worker]),:]

        # 希望休を調整納期表にコピー（希望休ですべて上書きして納期を一度消す）
        #index毎にadjusted_due_date_df.columns[4:]取得、但しカラムはadjusted_due_date_df
        #値はworker_off_days.valuesに入替え
        adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[5:]] = worker_off_days.values   #スキルグループ→工程による列追加

    # 割り当てたワーカーを adjusted_due_date_df へ「作業者」列として追加（※ここから adjusted_due_date_df が一列増えるので注意）
    adjusted_due_date_df.insert(3, "作業者", worker_list)
    #print('for後作業者追加adjusted_due_date_df',adjusted_due_date_df)
    # 作業者の希望休に合わせて、納期を調整
    after_index = None                                                                                                              # 後工程の index を保持する

    for index, row in adjusted_due_date_df[::-1].iterrows():                                                                        # due_date_df を逆順で行ごとに処理

        # 現工程の納期を抽出
        process: pd.Series = due_date_df.loc[index, due_date_df.columns[5:]]                                                        #スキルグループ→工程による列追加
        #process: pd.Series = due_date_df.loc[index, due_date_df.columns[4:]]                                                       # 現工程の納期表データ
        process_due_date = process[process.isin([3])]                                                                               # 現工程の納期
        #print("入力納期:", process_due_date.index[0])
        #print("入力納期process_due_date:", process_due_date)


        # 後工程の納期と本工程の納期を確認して調整
        # 後工程候補があるか（一つ目のデータは None なので無視）
        if after_index != None:
#adjusted_due_date_df["製造番号"]と#adjusted_due_date_df["製造番号"]が同じS?
            after_process_df: pd.DataFrame = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == row["製造番号"]]                               # 同じ製造番号のタスクを取り出す。

            #print("[INFO]: 工程:", row["工程"])

            # 後工程があるか確認（後工程候補　に　現工程の次のスキルグループ（工程）があるか確認する）

            #if row["スキルグループ"] + 1 in after_process_df["スキルグループ"].values:
            if row["工程"] + 1 in after_process_df["工程"].values:

                print("後工程あり")

                #after_index = after_process_df[after_process_df["スキルグループ"].isin([row["スキルグループ"] + 1])].index[0]　#スキルグループ→工程
                after_index = after_process_df[after_process_df["工程"].isin([row["工程"] + 1])].index[0]
                print("after_index :", after_index)

                # 後工程の納期を抽出
                after_process: pd.Series = adjusted_due_date_df.loc[after_index, adjusted_due_date_df.columns[6:]]                 #スキルグループ→工程による列追加

                after_process_due_date = after_process[after_process.isin([3])]                                                     # 後工程の納期
                #print("後工程の納期:", after_process_due_date.index[0])

                # 本工程の納期が後工程の納期より前に設定されていない場合　→　本工程の納期を後工程より前の日付にする
                if not process_due_date.index[0] < after_process_due_date.index[0]:

                    # 納期を１営業日早めて調整する（この時点では、有休をチェックしてないので決定しない）
                    before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[5]:after_process_due_date.index[0]]     # 現在の納期より前のデータを抽出
                    #before_days:pd.DataFrame = due_date_df.loc[[index], due_date_df.columns[4]:after_process_due_date.index[0]]  #スキルグループ→工程による列追加
                    new_due_date = before_days.columns[-2]                                                                          # 納期を１営業日早める
                    #print("後工程を考慮した納期調整:", new_due_date)

                    due_date_df.loc[index, process_due_date.index[0]] = 0                                                           # 現在の納期を納期表から削除
                    due_date_df.loc[index, new_due_date] = process_due_date.values[0]                                               # 新しい納期を納期表に追加

                    process: pd.Seies = due_date_df.loc[index, due_date_df.columns[5:]]                                            #スキルグループ→工程による列追加
                    #process: pd.Series = due_date_df.loc[index, due_date_df.columns[4:]]                                           # 現工程の納期表データ
                    process_due_date = process[process.isin([3])]                                                                   # 現工程の納期
                    print("現工程の納期:", process_due_date.index[0])

            else:

                print("後工程なし:")

        else:

            print("後工程なし（1つ目のデータ）:")


        # 希望休に合わせた調整
        # 現在の納期より前のデータを抽出
        before_days: pd.DataFrame = adjusted_due_date_df.loc[[index], adjusted_due_date_df.columns[6]:process_due_date.index[0]] #スキルグループ→工程による列追加

#X 納期を設定できるか確認していく→new_due_dateには6/12が入る？
        for new_due_date in before_days.columns[::-1]:

            # 納期を１日早める(-1日)
            print("納期候補:", new_due_date)

            # その日が希望休でない場合(0 のとき)　→　納期決定　→　ループ終了
            if adjusted_due_date_df.loc[index, new_due_date] == 0:

                # 納期が確定するのはココだけ。valuesで[3]が入るindexと配列指定。values[0]で3指定。
                print("納期決定:", new_due_date)
                adjusted_due_date_df.loc[index, new_due_date] = process_due_date.values[0]
                break

        after_index = index
        print("-" * 100)              # デバッグ用（ループごとに横線を表示）

    return adjusted_due_date_df


# %%
#受取
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

# %%
#第1世代

# %% [markdown]
# ## ※　データのルール
# 製造番号は必ず連番

# %%
adjusted_due_date_df[adjusted_due_date_df["製造番号"] == 1].iloc[:, :15]

# %% [markdown]
# ## 4.納期をランダムで振り直し（再帰処理）
# ※　この処理中は希望休を考慮しない
#

# %% [markdown]
# ### 開始日

# %% [markdown]
#

# %%
start_date = datetime.datetime(2023, 6, 2)

# %% [markdown]
# ### 納期のランダム振り分け関数

# %%
def generate_random_dates(
        start_date: datetime.datetime,
        end_date: datetime.datetime):

    start_date = due_date_df.loc[:, start_date:].columns[1]

    working_days = due_date_df.loc[:, start_date:end_date].columns
    #print(working_days)

    random_num = random.randint(0, working_days.shape[0]-1)
    random_date = working_days[random_num]
    #print(random_date)
    #print("-" * 50, "納期ランダム割り振り完了","-" * 50)

    return random_date

# %% [markdown]
# ### 同じ製造番号のタスクをランダムにリスケする関数（これが再帰関数：関数内の処理に自分自身の呼び出しがある）

# %%
def random_due_date(
        start_date: datetime.datetime,
        process_df: pd.DataFrame,
        process_num: int):

    #print("process_num :", process_num, "-" * 100)
    #print('process_df',process_df.loc[[process_df.index[process_num]], process_df.columns[:5]])

    process: pd.Series = process_df.loc[process_df.index[process_num], process_df.columns[5:]]              # 現工程の納期表データ
    deadline = process[process.isin([3])]                                                                   # 現工程の納期
    deadline_date: datetime.datetime = deadline.index[0]                                                    # その工程のデッドライン
    print("process_deadline :", deadline_date)

    # 最初の工程以外の時
    if process_num != 0:

        start_date = random_due_date(start_date, process_df.loc[process_df.index[:-1], :], process_num-1)   # 開始日を前工程の納期に置き換える（※　ここが再帰処理）

    new_due_date = generate_random_dates(start_date, deadline_date)                                         # 開始日～デッドライン　の期間からランダムに日付を選択

    due_date_df.loc[process_df.index[process_num], deadline.index[0]] = 0                                   # 現在の納期を納期表から削除
    due_date_df.loc[process_df.index[process_num], new_due_date] = deadline.values[0]                       # 新しい納期を納期表に追加

    return new_due_date



# %% [markdown]
# ### タスクを製造番号ごとに処理する関数

# %%
#def change_due_date(off_day_df, skill_df, due_date_df, product_num):
def change_due_date():

    serial_number = due_date_df.loc[:, "製造番号"].max()
    print('serial_number',serial_number)

    # 製造番号ごとに繰り返す
    for i in range(1, serial_number + 1):

        # 同じ製造番号のタスクを取り出す。
        process_df = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i]
        #練習
        #print('process_df',process_df)
        print('process_df.loc[:, process_df.columns].isin([3])')



        random_due_date(start_date, process_df, process_df.shape[0]-1)                                          # 製造番号ごとに納期のランダム割り当て（再帰関数）
        #練習
        print('タスク？',process_df)
        print('process_df.shape[0]',process_df.shape[0])
        print("*" * 200)

### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。



# %%
# 呼び出し
change_due_date()

# %%
#due_date_df

# %% [markdown]
# ### ※　この時点では、希望休を考慮した納期になっていない為、first_gen()関数を使ってもう一度希望休に沿った調整をする。

# %%
adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)

# %% [markdown]
# #### これでランダムな納期かつ希望休の調整ができる

# %% [markdown]
# ##### 希望休表

# %% [markdown]
# ##### 希望休調整前(ランダム)

# %% [markdown]
# ##### 希望休調整後

# %%
adjusted_due_date_df_copy= adjusted_due_date_df.copy()
adjusted_due_date_df_copy=adjusted_due_date_df_copy.iloc[:,5:]
adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
#行と列の数を取り出す
sh3=adjusted_due_date_df_copy.shape
adjusted_due_date_df_copy.columns=range(sh3[1])


# %%
#45変換
def pre_evalution(adjusted_due_date_df):
    adjusted_due_date_df2=adjusted_due_date_df.iloc[:,6:] #スキルグループ→工程による
    return adjusted_due_date_df2

# %%
#受取
adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)

# %%


# %% [markdown]
# ## 5.評価方法

# %%
##(1)工程ができるだけ長くならないこと
##(2)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。
#評価方法
def evalution_function(adjusted_due_date_df2):

##(3)前倒し生産方式の場合、できるだけ開始日に近いこと、ジャストインタイム方式の場合、できるだけ納期に近いこと。
#秒換算
# 'x-1
    score3=[]
    fix_day3= []
    tdos_list3=[]
    adjusted_due_date_df_copy= adjusted_due_date_df2.copy()
    adjusted_due_date_df_copy.reset_index(inplace=True,drop=True)
    #行と列の数を取り出す
    sh3=adjusted_due_date_df_copy.shape
    adjusted_due_date_df_copy.columns=range(sh3[1])
    print('sh3',sh3)
    print('sh3[1]',sh3[1])

        # 納期を１日早める(-1日)
        #print("納期候補:", new_due_date)
    for k in range(len(adjusted_due_date_df_copy)):
        for v in range(len(adjusted_due_date_df_copy.columns)):
            if adjusted_due_date_df_copy.iloc[k,v]==3:
                fix_day3.append(adjusted_due_date_df.columns[v+5])
                #fix_day.append(adjusted_due_date_df_copy.columns[v])
        # その日が希望休でない場合(0 のとfixき)　→　納期決定　→　ループ終了

    print('fix_day3',fix_day3)
    print(type(fix_day3))
    print('start_date',start_date)
    print(type(start_date))
    tdos3= '0000-00-00 00:00:00'
    tm3 : datetime.datetime
    #tm= tm.date()
    #for tm in range(len(fix_day)):
    for tm3 in fix_day3:
        print('tm',tm3)
        print(type(tm3))
        ttdelta3= tm3- start_date
        print('ttdelta3',ttdelta3)
        tdos3= ttdelta3.total_seconds()
        print('tdos3',tdos3)
        tdos3=int(tdos3)
        tdos_list3.append(tdos3)
    print('tdos_list3',tdos_list3)
    score3=sum(tdos_list3)*(-1)
    print('score3',score3)

#追加
##(4)工程ができるだけ長くならないこと
#工程間1日ずれ（品番==品番）len1=（日程（工程ｎ＋１）-（日程（工程ｎ）>0なら高得点
    # 'x-1
    score4=[]
    fix_day41= []
    fix_day42= []
    tdos_list4=[]
    serial_number = due_date_df.loc[:, "製造番号"].max()
    # 製造番号ごとに繰り返す
    for i in range(1, serial_number + 1):
        # 同じ製造番号のタスクを取り出す。
        during_process_df = adjusted_due_date_df[adjusted_due_date_df["製造番号"] == i]

        print('during_process_df',during_process_df)
        process_max = during_process_df.loc[:, "工程"].max()

        print('process_max',process_max)
        if process_max >1:

            for j in range(2, process_max):

                pds2:datetime= during_process_df[during_process_df["工程"]== j].iloc[0,:]

                pds20=pds2[pds2.isin([3])]

                if "工程" in pds20:
                    pass
                else:
                    pds20=pds20.index.values[0]

                    print('pds20',pds20)
                    #print('type(pds20)',type(pds20))

                    fix_day42.append(pds20)
                    print('fix_day42',fix_day42)
                    #print('type(fix_day42)',type(fix_day42))

                    tm42 : datetime.datetime
                    for tm42 in fix_day42:
                        print('tm42',tm42)
                        #print(type(tm42))


                    pds1:pd.Series= during_process_df[during_process_df["工程"]== j-1].iloc[0,:]
                    pds10=pds1[pds1.isin([3])]
                    pds10=pds10.index.values[0]
                    print('pds10',pds10)

                    fix_day41.append(pds10)
                    print('fix_day41',fix_day41)

                    tm41 : datetime.datetime
                    for tm41 in fix_day41:
                        print('tm41',tm41)
                        print(type(tm41))

                        ttdelta4= tm42- tm41
                        print('ttdelta4',ttdelta4)
                        tdos4= ttdelta4.total_seconds()
                        print('tdos4',tdos4)
                        tdos4=int(tdos4)
                        #追加
                        tdos4= tdos4- 86400
                        if tdos4== 0:
                            tdos4= tdos4- 100000
                        elif tdos4< 0:
                            tdos4=tdos4+ 100000
                        elif tdos4== 86400:
                            tdos4= tdos4-50000
                        tdos_list4.append(tdos4)
                        print('tdos_list4',tdos_list4)

        elif process_max ==1 :
            #何もしない
            pass

        score4=sum(tdos_list4)*(-1)
        print('score4',score4)




    score=  score3+score4


    return score


# %%
#受取
score= evalution_function(adjusted_due_date_df2)

# %% [markdown]
# ## 6.遺伝的アルゴリズム

# %%
##62遺伝的アルゴリズム


def crossover(ep,sd,p1,p2):
    #1か月の日数
    #days=len(p1.columns)
    days= len(off_day_df.columns) - 2
    #1次元化
    p1=np.array(p1).flatten()
    p2=np.array(p2).flatten()

    print(p1)
    print(p2)
    #子の変数
    ch1=[]
    ch2=[]
    for p1_,p2_ in zip(p1,p2):
        if ep > random.random():
            ch1.append(p1_)
            ch2.append(p2_)
        else:
            ch1.append(p2_)
            ch2.append(p1_)

    #突然変異
    ch1=mutation(sd, np.array(ch1).flatten())
    ch2=mutation(sd, np.array(ch2).flatten())

    #pandasに変換
    ch1=pd.DataFrame(ch1.reshape([int(len(ch1)/days), days]))
    ch2=pd.DataFrame(ch2.reshape([int(len(ch2)/days), days]))
    #列名の変更
    ch1.columns=[i+1 for i in range(len(ch1.columns))]
    ch2.columns=[i+1 for i in range(len(ch2.columns))]
    return ch1,ch2


#突然変異
def mutation(sd2, ch):
    if sd2 > random.random():

        rand = np.random.permutation(list(range(len(ch))))

        #遺伝子の10%を変異させる
        #並び換えた先頭10%
        rand=rand[:int(len(ch)//10)]
        for i in rand:
            #1なら0,0なら1
            if ch[i]==3:
                ch[i]==0

            if ch[i]==0:
                ch[i]==3
                #ch2も同じことをする。

    return ch


# %% [markdown]
# ## 7.全体

# %%
##72全体
#遺伝的アルゴリズム
#エクセル読み込み
off_day_df, skill_df, due_date_df, product_num , start_day =read_excels()
#親の保存
parent=[]
#for i in range(100):
#3世代へ変更 停止用
for i in range(3):
    #第1世代
    adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)
    #休日数の修正

    change_due_date()
    adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)


    #評価
    #追加
    adjusted_due_date_df2= pre_evalution(adjusted_due_date_df)
    score= evalution_function(adjusted_due_date_df2)
    #第1世代を格納

    parent.append([score, adjusted_due_date_df2.values]) #

#print('parent',parent)



#上位個体数
elite_length=20
#世代数
gene_length=50

#一様交叉確立
ep=0.5
#突然変異確立
sd=0.05



for i in range(gene_length):
    #点数で並び換え
    parent = sorted(parent, key=lambda x: -x[0])
    #parent=sorted(np.array(parent), key=lambda x:-x[0])
    #parent=sorted(np.array(parent[0]).flatten(), key=lambda x:-x[0])
    #p1=np.array(parent[0]).flatten()
    #上位個体を選別
    parent=parent[:elite_length]
    #各世代
    print('第'+str(i+1)+'世代')

    #最高得点の更新

    if i ==0:
        top=parent[0]
            #各世代の最高得点の表示
        print(top[0])
        print(np.array(top[1]))
    else:
        if top[0]>parent[0][0]:
            parent.append(top)
            #各世代の最高得点の表示
            print(top[0])
            print(np.array(top[1]))
        else:
            top=parent[0]
            #各世代の最高得点の表示
            print(parent[0][0])
            print(np.array(parent[0][1]))



    #子世代
    children=[]
    #遺伝子操作
    for k1,v1 in enumerate(parent):
        for k2,v2 in enumerate(parent):
            if k1<k2:
                #一様交叉
                print(ep)
                print(sd)
                print(v1[1])
                print(v2[1])
                ch,ch2=crossover(ep,sd,v1[1],v2[1])
                #休日数変更
                #ch21=noki_fix(ch21,product_num)
                #ch22=noki_fix(ch22,product_num)
                #納期をランダムで振り直し（再帰処理）
                #納期のランダム振り分け関数:def generate_random_date;受取りなし,def random_due_date;受取りなし,
                # 呼び出し?
                change_due_date()
                adjusted_due_date_df = first_gen(off_day_df, skill_df, due_date_df, product_num)
                #評価
                score1=evalution_function(ch)
                score2=evalution_function(ch2)
                #子孫を格納
                children.append([score1,ch])
                children.append([score2,ch2])

    #子を親にコピー
    parent=children.copy()

#最強個体の保存
x=top[1].replace(3,'☆').replace(2,'◎').replace(0,'')
x.to_excel('f_shiftf_4234short.xlsx')





Overwriting app4.py
Overwriting app4.py


In [24]:
#st内容はappへ記載。

In [25]:
from google.colab import files
#files.view("/content")
#files.view("app2.py")

# files.view("/content")
# files.view("app3.py")
files.view("/content")
files.view("app4.py")

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

<IPython.core.display.Javascript object>

In [26]:
#import streamlit as st
#st.write(hyouji_list)
#t.table(hyouji_list)

In [27]:
!streamlit config show


 # Below are all the sections and options you can have in
 ~/.streamlit/config.toml.
 

[global]

# By default, Streamlit checks if the Python watchdog module is available
# and, if not, prints a warning asking for you to install it. The watchdog
# module is not required, but highly recommended. It improves Streamlit's
# ability to detect changes to files in your filesystem.

# If you'd like to turn off this warning, set this to True.

# Default: false
# DEPRECATED.
# global.disableWatchdogWarning has been deprecated has been deprecated and will be removed in a future version.
# This option will be removed on or after 2024-01-20.
# disableWatchdogWarning = false

# By default, Streamlit displays a warning when a user sets both a widget
# default value in the function defining the widget and a widget value via
# the widget's key in `st.session_state`.

# If you'd like to turn off this warning, set this to True.

# Default: false
# disableWidgetStateDuplicationWarning = false

# If True

In [28]:
!streamlit run app4.py & sleep 3 && npx localtunnel --port 8501



npx: installed 22 in 3.43s

  You can now view your Streamlit app in your browser.

  Network URL: http://172.28.0.12:8501
  External URL: http://34.142.236.77:8501

your url is: https://famous-bars-ring.loca.lt
2024-01-04 19:51:01.722 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 534, in _run_script
    exec(code, module.__dict__)
  File "/content/app4.py", line 20, in <module>
    locale.setlocale(locale.LC_TIME, "ja_JP")   # 日本にローカライズ（これがないと、英語の曜日が出えてくる）
  File "/usr/lib/python3.10/locale.py", line 620, in setlocale
    return _setlocale(category, locale)
locale.Error: unsupported locale setting
2024-01-04 20:27:10.412 Uncaught app exception
Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/streamlit/runtime/scriptrunner/script_runner.py", line 534, in _run_script
    exec(code, module.__dict__)
  File "/content/app4.py", line 20, in <

In [29]:
!locale -a

C
C.utf8
en_US.utf8
POSIX
